<img src="images/banner.png" style="width: 100%;">

# Working With Different Data Formats

References:

[1] Visochek, Allan. *Practical Data Wrangling: Expert techniques for transforming your raw data into a valuable source for analytics.* Packt Publishing Ltd, 2017.

[2] McKinney, Wes. *Python for data analysis.* " O'Reilly Media, Inc.", 2022.

[3] pandas documentation - https://pandas.pydata.org/docs/

[4] Revised and grammar checked using ChatGPT - https://chatgpt.com/

Prepared by: Leodegario Lorenzo II

In [ ]:
import pandas as pd

Data encountered in real-world analysis varies in how structured it is - ranging from **structured** to **semi-structured** and **unstructured** formats. Preparing these data formats for analysis requires appropriate tools and data wrangling techniques. In this notebook, we explore how to work with structured and semi-structured data using Python, focusing on commonly used formats such as CSV, Excel, and JSON files. We will mainly use `pandas` and Python’s standard library as we we walk through fundamental data wrangling techniques.

## 1 Comma-Separated Values (CSV)

The comma-separated values (CSV) or `csv` format is a text file format used to store tabular data consisting of values separated by commas. Each data entry is represented by a single line and by convention contains column headers as its first row.

As an example of a `csv` file, we are given the different player statistics for the NBA Season 2024 to 2025 in `nba_24_25.csv`. A quick inspection of the file yields:

In [ ]:
!head data/nba_24_25.csv

The data is relatively clean with minimal errors in data entry. We can load this `csv` file into our notebook using the `pandas` function `read_csv` as a `DataFrame`.

In [ ]:
nba_24_25 = pd.read_csv('data/nba_24_25.csv')
nba_24_25

Notice that `pandas` automatically parses the first row in our `csv` as the column header. The index is also automatically created which corresponds to the row number of the data entry.

In [ ]:
nba_24_25.columns

In [ ]:
nba_24_25.index

Suppose that we want to use the player's name as index, we can set the index column by specifying the `index_col` parameter.

In [ ]:
nba_24_25 = pd.read_csv('data/nba_24_25.csv', index_col='Player')
nba_24_25

Some other parameters worth exploring and understanding for the `read_csv` function are - `delimeter`, `usecols`, `header`, `dtype`, `skiprows`, `nrows`, `na_values`, `encoding`, and `parse_dates`. See the documentation of **[pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)** for more details.

Let's demonstrate some essential `pandas` functionalities and a quick data analysis with the `csv` file provided.

Let’s begin by taking a look at the data types `pandas` inferred for each column.

In [ ]:
nba_24_25.dtypes

The data type inside each column has been properly inferred except for the `Data` column, which should have been a date time format. We can modify this by directly working on the data frame `nba_24_25`. But let's use the functionalities of `read_csv` to handle this siutation. We may also want to change the name of the `Data` column to be `Date` for it to be more informative.

In [ ]:
nba_24_25 = (pd.read_csv('data/nba_24_25.csv', index_col='Player',
                         parse_dates=['Data'])
               .rename(columns={'Data': 'Date'}))
nba_24_25

The `Data` column has now been properly parsed as a `datetime` format and renamed to `Date`.

In [ ]:
nba_24_25.dtypes

To select certain rows or subsets of the data, we can use the `loc` and `iloc` method of the `DataFrame`.

For example, to select the data rows of `Stephen Curry`:

In [ ]:
nba_24_25.loc['Stephen Curry', :]

If we only want specific columns, say the `3P` and `3PA`, we specify those columns as:

In [ ]:
nba_24_25.loc['Stephen Curry', ['3P', '3PA']]

You can also use a boolean array as your indexer. For example, we can select all instances where a player has scored more than 30 points in a game using the following expression:

In [ ]:
nba_24_25.loc[nba_24_25.PTS >= 30]

To sort the result, we can use the `sort_values` method specifying the proper parameters.

In [ ]:
nba_24_25.loc[nba_24_25.PTS >= 30].sort_values(by='PTS', ascending=False)

You can also chain different conditional statement using the *bitwise* operators.

In [ ]:
nba_24_25.loc[(nba_24_25.PTS >= 30) & (nba_24_25.Tm == 'GSW')].sort_values(by='PTS', ascending=False)

To create new columns, we can use `loc` then specify the desired new column as its label while performing an assigning operation for its value:

In [ ]:
nba_24_25.loc[:, 'Eff'] = nba_24_25.PTS / nba_24_25.MP

In [ ]:
nba_24_25.sort_values(by='Eff', ascending=False)

For a quick look of a several statistical measures for numerical data, we can use the `describe` function of `pandas.

In [ ]:
nba_24_25.PTS.describe()

Finally, we show how to perform a quick visualization of our data using `pandas`.

In [ ]:
ax = nba_24_25.MP.plot(kind='hist', bins=20)

ax.set_xlabel('Minutes Played')
ax.spines[['top', 'right']].set_visible(False)
ax.set_title("Histogram Plot of Minutes Played", weight='bold');

In [ ]:
nba_24_25.loc[nba_24_25.Date == '2024-12-25']

## 2 Excel Binary Format (XLS)

The main difference between Excel (`.xls` or `xlsx`) files and `csv` files is taht Excel files use a binary format, meaning they cannot be read directly as plain text. In terms of data structure, however, both formats typically store tabular data composed of rows and columns.

Unlike `csv` files, however, an Excel file can contain multiple worksheets within a single file. As such, pandas' `read_excel` function offers similar functionalities to `read_csv`, but also provides an additional option to specify which worksheet to load into a `DataFrame`.

To demonstrate some functionalities of pandas' `read_excel`, we will look at an Excel file containing the population data of NCR for May 2020.

In [ ]:
ncr = pd.read_excel('data/ncr.xlsx')
ncr

Notice that by default, `read_excel` loads the first sheet in the file. If we want to specifically open a certain sheet, we can specify it through the `sheet_name` parameter.

For now, let's work with the first sheet found by `read_excel`. This contains the population count for each city in NCR. However, there's some effort of data cleaning to be made.

In essence, what we want are the province, city, and municipality names and their corresponding population, thus we will drop some columns. We can accomplish this in several ways, we will show one such approach:

In [ ]:
ncr = (pd.read_excel('data/ncr.xlsx', skiprows=4, usecols=[1, 2],
                     names=['Province, City, and Municipality', 'Population'])
         .dropna().reset_index(drop=True))
ncr

We can export this data frame into an Excel file using the `DataFrame` method `to_excel`.

In [ ]:
ncr.to_excel('data/ncr_cleaned.xlsx', index=False)

If we want to write an Excel file with several sheets, we can use pandas' `ExcelWriter` using the `with` statement as a context manager.

Let's say for example, we want to separate the data into separate sheets for each administrative level (province, city, municipality). We can accomplish this using the following code:

In [ ]:
ncr.columns = ['Name', 'Population']
with pd.ExcelWriter('data/ncr_sheets.xlsx') as writer:
    ncr.loc[ncr.iloc[:, 0].str.contains('REGION')].to_excel(writer, sheet_name='region', index=False)
    ncr.loc[ncr.iloc[:, 0].str.contains('CITY')].to_excel(writer, sheet_name='city', index=False)
    ncr.loc[ncr.iloc[:, 0].str.contains('PATEROS')].to_excel(writer, sheet_name='municipality', index=False)

## 3 JavaScript Object Notation (JSON)

The `JSON` format is an example of a **semi-structured**, **hierarchical** data format. Compared to tabular data, `JSON` is more flexible nad free-form, though it often consists of a collection of records that follow a consistent internal structure. One of the main challenges in working with `JSON` files is understanding this structure and identifying which fields are relevant for extraction. While inspecting the `JSON` file directly can help in smaller examples, this approach becomes quickly impractical as file size and complexity increases.

In this section, we demonstrate two approaches for working with JSON files. THe first uses Python's built-in `json` library, while the other approach loads the JSON file directly into `pandas`. The `json` library is especially useful when the structure of the data is unclear or deeply nested, whereas `pandas` is often suitable for JSON files that already resemble tabular data.

### `json` library

In [ ]:
import json

One thing that we can notice is that JSON text files resembles Python dictionaries.

In [ ]:
json_str = """{ 
  "firstName": "John", 
  "lastName": "Smith", 
  "isAlive": true, 
  "age": 27, 
  "address": { 
    "streetAddress": "21 2nd Street", 
    "city": "New York", 
    "state": "NY", 
    "postalCode": "10021-3100" 
  }, 
  "phoneNumbers": [ 
    { 
      "type": "home", 
      "number": "212 555-1234" 
    }, 
    {
      "type": "office", 
      "number": "646 555-4567" 
    } 
  ], 
  "children": [], 
  "spouse": null 
}
"""

As such, the way we explore `json` files is similar to how we explore nested `dict` in Python. We can do so by first reading a json formatted string using `json.loads`

In [ ]:
json_as_dict = json.loads(json_str)
json_as_dict

In [ ]:
type(json_as_dict)

Now, usually `json` files don't come directly as string objects in Python, but are rather read as `.json` text files.

To demonstrate how we read `json` files, we will use a dataset from **Seeclickfix**, a platform to report non-emergency issues to local governments. This dataset contains a series of data entries that represents issue reports using the platform. The sample dataset is saved as `scf_data.json` in the `data` folder.

In [ ]:
with open('data/scf_data.json') as f:
    data = json.load(f)

Let's first understand the structure of the data by looking at the keys of this dictionary.

In [ ]:
data.keys()

It has three keys - `errors`, `issues`, and `metadata`. Among these, the core data lies in the `issues` key, which makes sense since the dataset contains issue reports. Let's look at what it contains.

In [ ]:
data['issues']

The issues is a list of data entries represented by Python dictionary. Which we can convert to a `DataFrame` for easier manipulation.

In [ ]:
df_issues = pd.DataFrame(data['issues'])

We can inspect a single element for a closer inspection of the data structure:

In [ ]:
df_issues.iloc[0]

Depending on the certain analysis that we want to do, we may want to select a subset of the data for further analysis. For this case, say we are concerned with how the `description` of the issue is related with the corresponding `rating` of the user and their location. We thus select columns `created_at`, `address`, `description`, `lat`, `lng`, `rating`.

In [ ]:
select_cols = ['created_at', 'address', 'description', 'lat', 'lng', 'rating']

df_scf = df_issues.loc[:, select_cols]
df_scf

Once again, let's check for the data types of the columns of our data frame. This time, we demonstrate the use of the `.info` method of `DataFrames`.

In [ ]:
df_scf.info()

Let's convert the `created_at` column into a datetime format.

In [ ]:
df_scf.created_at = pd.to_datetime(df_scf.created_at)

In [ ]:
df_scf.dtypes

Now, let's visualize some simple statistics on our data such as:

1. A frequency count of ratings.
2. A frequency count of issues having `trash` in their description.
3. A frequency count showing the time in which the issue was created at the platform.

In [ ]:
df_scf.rating.value_counts()

In [ ]:
df_scf.description.str.lower().str.contains('trash').value_counts()

In [ ]:
df_scf.created_at.dt.hour.value_counts()

### pandas `read_json`

If the `json` text file already resembles a tabular structure, we can try to directly load it using pandas' function `read_json`.

As an example, let's look at a sample twitter data uploaded as the json file `sample_twitter.json.bz2`. We inspect it first using Python's standard libraries. Here, we will need the `bz2` library to decompress the json file.

In [ ]:
import bz2

In [ ]:
with bz2.open('data/sample_twitter.json.bz2', 'r') as f:
    contents = f.readlines()

In [ ]:
print(contents[:10_000])

Notice here that the data entry for this file is done per line, wherein each line follows the `json` format. In this case, if we want to use the `json` library to load the data, we will perform a list comprehension.

In [ ]:
list_of_json = [json.loads(line) for line in contents]

Which we can then again convert directly to a DataFrame.

In [ ]:
pd.DataFrame(list_of_json)

Since the raw data already resemebles some data structure, we can use `read_json` directly. However, we specify `lines` as `True`, since as we discovered, the data consists of lines of `json` text.

In [ ]:
df_twitter = pd.read_json('data/sample_twitter.json.bz2', lines=True, nrows=10)

### Writing `json` files using `pandas`

Given any data frame, we can export it into a `json` format in several ways:

- ‘split’ : dict like {{‘index’ -> [index], ‘columns’ -> [columns], ‘data’ -> [values]}}

- ‘records’ : list like [{{column -> value}}, … , {{column -> value}}]

- ‘index’ : dict like {{index -> {{column -> value}}}}

- ‘columns’ : dict like {{column -> {{index -> value}}}}

- ‘values’ : just the values array

- ‘table’ : dict like {{‘schema’: {{schema}}, ‘data’: {{data}}}}

In [ ]:
df_twitter

In [ ]:
print(df_twitter.to_json(orient='split', indent=2))

In [ ]:
print(df_twitter.to_json(orient='records', indent=2))

In [ ]:
print(df_twitter.to_json(orient='index', indent=2))

In [ ]:
print(df_twitter.to_json(orient='columns', indent=2))

In [ ]:
print(df_twitter.to_json(orient='values', indent=2))

In [ ]:
print(df_twitter.to_json(orient='table', indent=2))

`to_json` outputs a string, as such to save it onto a text file, we need to write it to a file using Python.

In [ ]:
with open('data/sample_tweets.json', 'w') as f:
    f.write(df_twitter.to_json(orient='records', indent=2))

That's it folks! We have shown how to load and wrangle data from CSV, Excel, and JSON files using Python and `pandas`, and how these steps enable basic data analysis. Mastering these workflows provides a strong foundation for working with real-world datasets and sets the stage for more advanced analytical techniques.

<img src="images/banner-down.png" style="width: 100%;">